In [3]:
import sys
sys.path.append('../..')
from config import client

from pymongo import UpdateOne
from tqdm import tqdm

from mlpp.data_collection.sample import ScoresSubset

In [4]:
osu_db = client['osu_random_db']

In [5]:
sample_3k = ScoresSubset(osu_db['scores_sample_3k'], osu_db['users_sample_3k'])

In [8]:
def compute_acc(score):
    c50, c100, c300, cmiss = score['count50'], score['count100'], score['count300'], score['countmiss']
    return (c50 / 6 + c100 / 3 + c300) / (c50 + c100 + c300 + cmiss)

In [9]:
def compare(s_i, s_j):
    return compute_acc(s_i) < compute_acc(s_j)

In [10]:
fields = {
    'count50': 1,
    'count100': 1,
    'count300': 1,
    'countmiss': 1,
    'beatmap_id': 1,
    'user_id': 1
}

In [23]:
count_fail = 0
def beatmap_comparisons(user_id):
    ordered_scores = list(sample_3k.scores_high.find({'user_id': user_id}, fields).sort('date'))
    comparisons = []
    global count_fail 
    
    for i in range(len(ordered_scores) - 1):
        s_i, s_j = ordered_scores[i], ordered_scores[i + 1]
        
        try: 
            i, j = s_i['beatmap_id'], s_j['beatmap_id']

            obsv = [i, j] if compare(s_i, s_j) else [j, i]
            comparisons.append(obsv)
        except: 
            count_fail += 1
    
    return comparisons

In [12]:
user_ids = [u['_id'] for u in sample_3k.user_stats.find({}, {'_id': 1})]

In [24]:
comps = []
for user_id in tqdm(user_ids):
    comps.extend(beatmap_comparisons(user_id))

100%|██████████| 3000/3000 [16:38<00:00,  3.00it/s]
